In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import csv
import sys
import os
import seaborn as sns
import pandas as pd

In [36]:
Ncolors = 9
colormap = plt.cm.viridis  # LinearSegmentedColormap
Ncolors = min(colormap.N, Ncolors)
mapcolors = [colormap(int(x * colormap.N / Ncolors)) for x in range(Ncolors)]

In [13]:
def load_pd_positions(dirPath, experiment_type):
    if experiment_type != "experiment" and experiment_type != "baseline":
        print("experiment_type could be only $experiment or $baseline")
        exit(-1)

    num_experiment = len([name for name in os.listdir(dirPath) if
                          (os.path.isfile(os.path.join(dirPath, name)) and (name.endswith('position.tsv')))])

    if os.path.exists(dirPath + "/" + experiment_type + ".pkl"):
        return num_experiment, pd.read_pickle(dirPath + "/" + experiment_type + ".pkl")
    # else:
    #     print("Baseline:"+dirPath+" not an existing path")
    #     exit(-1)

    print("Generating pickle positions file in " + dirPath + "/" + experiment_type + ".pkl")
    df = pd.DataFrame()
    for filename in os.listdir(dirPath):
        if filename.endswith('positions_file.tsv'):
            if not os.path.getsize(os.path.join(dirPath, filename)) > 0:
                print("Error, empty file at:" + os.path.join(dirPath, filename))
                continue
        df_single = pd.read_csv(dirPath + "/" + filename, sep="\t")
        df = df.append(df_single)

    df.to_pickle(dirPath + "/" + experiment_type + ".pkl")
    return num_experiment, df

In [14]:
folder = "/home/luigi/Documents/ARGoS/argos_results_scripts/esperimentino"
[number_of_experiments, df_experiment] = load_pd_positions(folder, "experiment")

### Baseline, 1 sec straight 1 sec left and right rotation

In [337]:
df_base = pd.DataFrame()
for filename in os.listdir(folder):
    if filename.endswith('positions_file.tsv'):
        continue
        
    df_base = pd.read_csv(folder + "/" + filename, sep="\t") 

In [338]:
base_concatenated = df_base.values[:, 1:]
base_concatenated = np.array([x.split(',') for x in base_concatenated.ravel()], dtype=float)
base_concatenated = base_concatenated.reshape(1,-1,2)
base_concatenated = np.append(base_concatenated,base_concatenated, axis=0)

In [339]:
base_concatenated.shape

(2, 901, 2)

In [341]:
df = pd.DataFrame()
for filename in os.listdir(folder):
    straight_millisec = -1
    turn_millisec = -1
    
    if filename.endswith('positions_file.tsv'):
        print(filename)
        elements = filename.split("_")
        for e in elements:
            if e.startswith("straight"):
                straight_millisec = e.split("#")[-1]
            if e.startswith("turn"):
                turn_millisec = e.split("#")[-1]
    
    else:
        continue
    
    print("Straight:",straight_millisec)
    print("Turning:",turn_millisec)
    
    df = pd.read_csv(folder + "/" + filename, sep="\t")
    positions_concatenated = df.values[:,1:]
    positions_concatenated = np.array([x.split(',') for x in positions_concatenated.ravel()],dtype=float)
    positions_concatenated = positions_concatenated.reshape(1,-1,2)
    positions_concatenated = np.append(positions_concatenated,positions_concatenated, axis=0)
    
    total_experiment_wmsd = []
    baseline_experiment_wmsd = []

    for window_size in range(1, 10):
        w_displacement_array = np.array([])
        base_w_displacement_array = np.array([])

        win_disp = fixed_window_displacement(positions_concatenated, window_size)
        base_win_disp = fixed_window_displacement(base_concatenated, window_size)


        w_displacement_array = np.vstack(
            [w_displacement_array, win_disp]) if w_displacement_array.size else win_disp

        base_w_displacement_array = np.vstack(
            [base_w_displacement_array, base_win_disp]) if base_w_displacement_array.size else base_win_disp

        total_experiment_wmsd.append(w_displacement_array)
        baseline_experiment_wmsd.append(base_w_displacement_array)
        
    plot_both_wmsd(baseline_experiment_wmsd, total_experiment_wmsd, straight_millisec, turn_millisec)

straight#2000_turn#3000_positions_file.tsv
Straight: 2000
Turning: 3000
Storing fig: comparison_rotation_straight_2000_turning_3000_seconds.png
straight#2000_turn#100_positions_file.tsv
Straight: 2000
Turning: 100
Storing fig: comparison_rotation_straight_2000_turning_100_seconds.png
straight#2000_turn#500_positions_file.tsv
Straight: 2000
Turning: 500
Storing fig: comparison_rotation_straight_2000_turning_500_seconds.png
straight#1000_turn#2000_positions_file.tsv
Straight: 1000
Turning: 2000
Storing fig: comparison_rotation_straight_1000_turning_2000_seconds.png
straight#2000_turn#1500_positions_file.tsv
Straight: 2000
Turning: 1500
Storing fig: comparison_rotation_straight_2000_turning_1500_seconds.png
straight#2000_turn#2000_positions_file.tsv
Straight: 2000
Turning: 2000
Storing fig: comparison_rotation_straight_2000_turning_2000_seconds.png
straight#2000_turn#1000_positions_file.tsv
Straight: 2000
Turning: 1000
Storing fig: comparison_rotation_straight_2000_turning_1000_seconds.pn

(2, 901, 2)

In [301]:
minor_straight_concatenated = df_base.values[:, 1:]
minor_straight_concatenated = np.array([x.split(',') for x in minor_straight_concatenated.ravel()], dtype=float)
minor_straight_concatenated = minor_straight_concatenated.reshape(1,-1,2)
minor_straight_concatenated = np.append(minor_straight_concatenated,minor_straight_concatenated, axis=0)

In [302]:
minor_straight_concatenated.shape

(2, 901, 2)

In [303]:
positions_concatenated = minor_straight_concatenated

### Qui prendi posizione per posizione !!!!!!!!!!!!!!!!!!!!!!

In [304]:
pos = 6
positions_concatenated = df.values[:,1:] 

In [279]:
positions_concatenated = np.array([x.split(',') for x in positions_concatenated.ravel()],dtype=float)
positions_concatenated = positions_concatenated.reshape(1,-1,2)
positions_concatenated = np.append(positions_concatenated,positions_concatenated, axis=0)

In [305]:
total_experiment_wmsd = []
baseline_experiment_wmsd = []

for window_size in range(1, 10):
    w_displacement_array = np.array([])
    base_w_displacement_array = np.array([])
    
    win_disp = fixed_window_displacement(positions_concatenated, window_size)
    base_win_disp = fixed_window_displacement(base_concatenated, window_size)
    
    
    w_displacement_array = np.vstack(
        [w_displacement_array, win_disp]) if w_displacement_array.size else win_disp
    
    base_w_displacement_array = np.vstack(
        [base_w_displacement_array, base_win_disp]) if base_w_displacement_array.size else base_win_disp

    total_experiment_wmsd.append(w_displacement_array)
    baseline_experiment_wmsd.append(base_w_displacement_array)

In [323]:
def fixed_window_displacement(df, window_size):
    w_displacement_matrix = np.array([])
    for f in range(window_size, df.shape[1], window_size):
#         print("f:", f)
        tf = df[:, f]
        ti = df[:, f - window_size]
        sq_distance = np.sum((tf - ti) ** 2, axis=1)
        wmsd = np.true_divide(sq_distance, window_size ** 2)
        w_displacement_matrix = np.column_stack([w_displacement_matrix, wmsd]) if w_displacement_matrix.size else wmsd

    w_displacement_array = np.mean(w_displacement_matrix, axis=0)
    return w_displacement_array

In [257]:
len(total_experiment_wmsd)

9

In [307]:
seconds = "2_straight_0.5_sec"

In [312]:
plot_both_wmsd(baseline_experiment_wmsd, total_experiment_wmsd)

In [334]:
def plot_both_wmsd(base_matrix, total_wmsd_matrix, straight_millisec, turn_millisec, storage_dir='./comparison'):
    fig = plt.figure(figsize=(20, 10), dpi=160, facecolor='w', edgecolor='k')
    
    for i, y in enumerate(total_wmsd_matrix):
        times = np.linspace(0, len(y) * (i + 1) * 10, len(y), endpoint=True)

        #         print("indice finestra: ", i+1)
        #         print("numero di punti: ", len(y))
        #         print("Max val linspace: ", len(y)*(i+1)*10)
        #         print("times shape:", times.shape)
        #         print(times)
        plt.plot(times, y, label=i + 1, marker='o', color=mapcolors[i])

    for i, y in enumerate(base_matrix):
        times = np.linspace(0, len(y) * (i + 1) * 10, len(y), endpoint=True)

        plt.plot(times, y, label="b" + str(i + 1), linestyle='dashed', alpha=0.6, color=mapcolors[i])

    fig.legend(loc=7, bbox_to_anchor=(0.95, 0.5))  # , prop={'size': 20})
    #     fig.subplots_adjust(right=0.9)
    #     plt.show()

    plt.title("WMSD with different w_size, going straight for "+str(straight_millisec)+" milliseconds and turning for "+str(turn_millisec)+" milliseconds")
    plt.ylabel('WMSD')
    plt.xlabel('time(s)')
    #     plt.legend(loc='lower right')
#     plt.xticks(np.arange(0, 9000, 200))
    #     plt.grid(which='minor')
    plt.grid()
#     plt.ylim((0.0085, 0.0095))

    # ax = plt.axes()
    # #     plt.setp(ax.get_xticklabels(),visible=False)
    #     # Make a plot with major ticks that are multiples of 20 and minor ticks that
    #     # are multiples of 5.  Label major ticks with '%d' formatting but don't label
    #     # minor ticks.
    #     ax.xaxis.set_major_locator(MultipleLocator(100))
    #     ax.xaxis.set_major_formatter(FormatStrFormatter('%d'))

    #     # For the minor ticks, use no labels; default NullFormatter.
    #     ax.xaxis.set_minor_locator(MultipleLocator(10))
    #     ax.yaxis.set_minor_locator(AutoMinorLocator(2))
    #     ax.yaxis.set_major_formatter(FormatStrFormatter('%f'))

    #     plt.show();
    fileName = "comparison_rotation_straight_"+str(straight_millisec)+"_turning_"+str(turn_millisec)+"_seconds.png" 
    print("Storing fig:",fileName)
    plt.savefig(storage_dir + '/' + fileName)
    plt.close(fig)
